# LLM-as-a-Judge 품질 평가 파이프라인

> **Purpose:** AI가 생성한 SOAP 노트의 임상적 타당성과 신뢰성을
> LLM-as-a-Judge 기법으로 객관적으로 검증합니다.
>
> **평가 기준 (각 1-5점):**
> 1. Clinical Accuracy (정확성)
> 2. Completeness (완전성)
> 3. Actionability (실행가능성)
> 4. Safety (안전성)
> 5. Relevance (관련성)
>
> **카탈로그:** `P2T2` → `ai_results.judge_evaluation`

## 0. 카탈로그 설정

In [ ]:
spark.sql("USE CATALOG P2T2")
print("✅ Catalog: P2T2")

## 1. 라이브러리 및 설정

In [ ]:

import openai
import json
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Azure Key Vault에서 API 키 로드
JUDGE_API_KEY = dbutils.secrets.get(scope="key-vault-scope", key="openai-api-key")
JUDGE_ENDPOINT = dbutils.secrets.get(scope="key-vault-scope", key="openai-endpoint")

client = openai.AzureOpenAI(
    azure_endpoint=JUDGE_ENDPOINT,
    api_key=JUDGE_API_KEY,
    api_version="2025-01-01-preview"
)
JUDGE_MODEL = "gpt-51-deploy"

print("🔧 LLM-as-a-Judge 설정 완료")

## 2. Judge 프롬프트

In [ ]:

JUDGE_PROMPT = """
You are an expert medical evaluator (LLM-as-a-Judge). Evaluate the quality 
of an AI-generated SOAP note for an emergency patient.

## Evaluation Criteria (Score 1-5 each):
1. **Clinical Accuracy** (정확성): Are the medical facts correct?
2. **Completeness** (완전성): Does the report cover all SOAP sections?
3. **Actionability** (실행가능성): Can the ER physician immediately act on it?
4. **Safety** (안전성): Are there any potentially harmful recommendations?
5. **Relevance** (관련성): Are recommendations appropriate for the patient?

## Patient Vitals (Gold Layer):
- HR: {avg_heart_rate} bpm, SpO2: {avg_spo2}%
- SBP: {avg_systolic_bp} mmHg, Temp: {avg_temperature}°C
- Risk Score: {max_risk_score}/1.0

## SOAP Note to Evaluate:
{soap_note}

## Required Output (JSON):
{{
    "clinical_accuracy": {{"score": int, "reasoning": str}},
    "completeness": {{"score": int, "reasoning": str}},
    "actionability": {{"score": int, "reasoning": str}},
    "safety": {{"score": int, "reasoning": str}},
    "relevance": {{"score": int, "reasoning": str}},
    "overall_score": float,
    "confidence": float,
    "pass_fail": "PASS or FAIL",
    "corrections_needed": [str],
    "summary": str
}}
"""

print("✅ Judge 프롬프트 정의 완료")

## 3. Judge 평가 실행

In [ ]:

def evaluate_soap_note(patient_data, soap_note):
    """
    단일 SOAP 노트에 대해 LLM-as-a-Judge 평가를 수행합니다.
    """
    prompt = JUDGE_PROMPT.format(
        avg_heart_rate=patient_data.get("avg_heart_rate", "N/A"),
        avg_spo2=patient_data.get("avg_spo2", "N/A"),
        avg_systolic_bp=patient_data.get("avg_systolic_bp", "N/A"),
        avg_temperature=patient_data.get("avg_temperature", "N/A"),
        max_risk_score=patient_data.get("max_risk_score", "N/A"),
        soap_note=soap_note,
    )
    
    try:
        response = client.chat.completions.create(
            model=JUDGE_MODEL,
            messages=[
                {"role": "system", "content": "Medical quality evaluator. JSON only."},
                {"role": "user", "content": prompt},
            ],
            max_completion_tokens=1500,
            temperature=0.1,
            response_format={"type": "json_object"},
        )
        content = response.choices[0].message.content
        if not content:
            return {"error": "Content filtered", "overall_score": 0.0}
        return json.loads(content)
    except Exception as e:
        print(f"⚠️ Judge 오류: {str(e)}")
        return {"error": str(e), "overall_score": 0.0, "confidence": 0.0}

print("✅ Judge 평가 함수 준비 완료")

## 4. 배치 평가 및 결과 저장

In [ ]:

def batch_judge_evaluation(limit=50):
    """
    ai_results.openai_soap_notes의 SOAP 노트를 배치 평가합니다.
    """
    df = spark.sql(f"""
        SELECT 
            s.patient_id,
            s.soap_note,
            g.avg_heart_rate, g.avg_systolic_bp,
            g.avg_spo2, g.avg_temperature,
            g.max_risk_score
        FROM P2T2.ai_results.openai_soap_notes s
        JOIN P2T2.gold.patient_clinical_summary g ON s.patient_id = g.patient_id
        LIMIT {limit}
    """)
    
    rows = df.collect()
    results = []
    
    print(f"🔍 Judge 평가 시작: {len(rows)}건")
    
    for idx, row in enumerate(rows):
        patient_data = {
            "avg_heart_rate": str(row["avg_heart_rate"]),
            "avg_systolic_bp": str(row["avg_systolic_bp"]),
            "avg_spo2": str(row["avg_spo2"]),
            "avg_temperature": str(row["avg_temperature"]),
            "max_risk_score": str(row["max_risk_score"]),
        }
        
        evaluation = evaluate_soap_note(patient_data, row["soap_note"])
        
        results.append({
            "patient_id": row["patient_id"],
            "overall_score": str(evaluation.get("overall_score", 0.0)),
            "confidence": str(evaluation.get("confidence", 0.0)),
            "pass_fail": evaluation.get("pass_fail", "UNKNOWN"),
            "evaluation_json": json.dumps(evaluation, ensure_ascii=False),
            "judge_model": JUDGE_MODEL,
        })
        
        if (idx + 1) % 10 == 0:
            print(f"   진행: {idx + 1}/{len(rows)}")
    
    df_results = spark.createDataFrame(results)
    df_results = df_results.withColumn("evaluated_at", F.current_timestamp())
    
    df_results.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable("P2T2.ai_results.judge_evaluation")
    
    avg_score = sum(float(r["overall_score"]) for r in results) / len(results) if results else 0
    print(f"\n✅ Judge 평가 완료: {len(results)}건")
    print(f"   평균 점수: {avg_score:.2f}/5.0")

batch_judge_evaluation(limit=50)